# 训练模型

In [ ]:
# import lib
import torchvision
from torch.utils.tensorboard import SummaryWriter

# import  LeNet model
from model import *

from torch import nn
from torch.utils.data import DataLoader

下载数据集 train_data test_data

In [ ]:
# downdload dataset
# download dataset when "download=True"
train_data = torchvision.datasets.CIFAR10(
    "./CIFAR10",
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)

test_data = torchvision.datasets.CIFAR10(
    "./CIFAR10",
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True,
)

In [ ]:
# length
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

In [ ]:
# DataLoader
# batch_size=64
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
# create model
model = Model()
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
# loss function
loss_fn = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

In [ ]:
# optimizer
# SGD:Stochastic Gradient Descent
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# train step
total_train_step = 0
# test step
total_test_step = 0
# train epoch
epoch = 10

# add tensorboard
writer = SummaryWriter("logs_train")

## 开始训练

GPU训练更改部分：
1. 数据集
2. 损失函数
3. 优化器

In [ ]:
# start train
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i + 1))

    model.train()
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)

        # optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        # 每100次显示一次
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # start test
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))

    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)

    total_test_step = total_test_step + 1

    torch.save(model, "./model/CIFAR10_{}.pth".format(i))
    print("模型已保存")

writer.close()